In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils import plot_model
import os,math,random,cv2
from keras import optimizers
import keras.backend as K
import keras
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

Using TensorFlow backend.


In [2]:
def split_train_test(percent=0.8):
    categories=[f for f in os.listdir('data') if not f.startswith('.')]
    for c in categories:
        allImages=[f for f in os.listdir('data/'+c) if f.endswith('.jpg') and not f.startswith('.')]
        totalImages=int(len(allImages))
        train_size=int(math.ceil(totalImages*percent))
        test_size=totalImages-train_size
        random.shuffle(allImages)
        train_images=allImages[:train_size]
        test_images=[item for item in allImages if item not in train_images]
        if not os.path.exists('./data/'+c+'/train'):
            os.makedirs('./data/'+c+'/train')
        if not os.path.exists('./data/'+c+'/test'):
            os.makedirs('./data/'+c+'/test')
        print(c+":"+len(train_images))
        for i in train_images:
            os.rename('./data/'+c+'/'+i,'./data/'+c+'/train'+'/'+i)
        for i in test_images:
            os.rename("./data/"+c+'/'+i,"./data/"+c+'/test'+'/'+i)
    return categories    

split_train_test()           

451
330
244
291
162
444
491
540


['bed', 'lamp', 'wardrobe', 'stool', 'shelf', 'chair', 'table', 'sofa']

In [3]:
categories=[f for f in os.listdir('data') if not f.startswith('.')]

In [4]:
def returnCategory(n):
    return categories[n]
def returnCatPos(x):
    return categories.index(x)

In [23]:
def load_data(resize_dim):
    X_train=[]
    Y_train=[]
    X_test=[]
    Y_test=[]
    for c in categories:
        print("Loading : "+c)
        train_folder='./data/'+c+'/train/'
        test_folder='./data/'+c+'/test/'
        for i in os.listdir(train_folder):            
            my_image=cv2.imread(train_folder+i)
            resized_image = cv2.resize(my_image, (resize_dim, resize_dim)) 
            X_train.append((resized_image))
            Y_train.append(returnCatPos(c))
            
        for i in os.listdir(test_folder):            
            my_image=cv2.imread(test_folder+i)
            resized_image = cv2.resize(my_image, (resize_dim, resize_dim)) 
            X_test.append((resized_image))
            Y_test.append(returnCatPos(c)) 
    X_train=np.asarray(X_train)
    Y_train=np.asarray(Y_train)
    X_test=np.asarray(X_test)
    Y_test=np.asarray(Y_test)
    return X_train,Y_train,X_test,Y_test
X_train,Y_train,X_test,Y_test=load_data(128)

Loading : bed
Loading : lamp
Loading : wardrobe
Loading : stool
Loading : shelf
Loading : chair
Loading : table
Loading : sofa


In [29]:
#Encode Y to one-hot
def encode_y():
    b = np.zeros((len(Y_train), 8))
    b[np.arange(len(Y_train)), Y_train] = 1
    Y_train_enc=b

    c = np.zeros((len(Y_test), 8))
    c[np.arange(len(Y_test)), Y_test] = 1
    Y_test_enc=b
    print(Y_test.shape)
encode_y()
print(Y_train_enc.shape)

(733,)
(2953, 8)


In [25]:
print("Number of training samples =",X_train.shape[0])
print("Number of testing samples =",X_test.shape[0])
print("Image dimension =",X_train.shape[1:])
print(Y_train.shape)

Number of training samples = 2953
Number of testing samples = 733
Image dimension = (128, 128, 3)
(2953,)


In [ ]:
# batch_size = 32
# samples_per_epoch = 1000
# nb_filters1 = 32
# nb_filters2 = 64
# conv1_size = 3
# conv2_size = 2
# pool_size = 2
# classes_num = 8
# lr = 0.0004

In [ ]:
# model = Sequential()
# model.add(Convolution2D(nb_filters1, conv1_size, conv1_size, border_mode ="same", input_shape=(64, 64, 3)))
# model.add(Activation("relu"))
# model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

# model.add(Convolution2D(nb_filters2, conv2_size, conv2_size, border_mode ="same"))
# model.add(Activation("relu"))
# model.add(MaxPooling2D(pool_size=(pool_size, pool_size), dim_ordering='th'))

# model.add(Flatten())
# model.add(Dense(256))
# model.add(Activation("relu"))
# model.add(Dropout(0.5))
# model.add(Dense(classes_num, activation='softmax'))

# model.compile(loss='categorical_crossentropy',
#               optimizer=optimizers.RMSprop(lr=lr),
# metrics=['accuracy'])
# # model.fit_generator(
# #     train_generator,
# #     samples_per_epoch=samples_per_epoch,
# #     epochs=epochs,
# #     validation_data=validation_generator,
# #     callbacks=cbks,
# #     validation_steps=validation_steps)

# # target_dir = './models/'
# # if not os.path.exists(target_dir):
# #   os.mkdir(target_dir)
# # model.save('./models/model.h5')
# # model.save_weights('./models/weights.h5')
# model.summary()

In [26]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=X_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(8, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.0001),
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 124, 124, 32)      2432      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 58, 58, 64)        51264     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 29, 29, 64)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 53824)             0         
_________________________________________________________________
dense_13 (Dense)             (None, 1000)              53825000  
_________________________________________________________________
dense_14 (Dense)             (None, 8)                 8008      
Total para

In [27]:
model.fit(x=X_train, y=Y_train_enc,batch_size=32,epochs=10,verbose=1)

Epoch 1/10
2953/2953 [==============================] - 182s 62ms/step - loss: 13.0376 - acc: 0.1863
Epoch 2/10
2953/2953 [==============================] - 192s 65ms/step - loss: 12.8261 - acc: 0.1991
Epoch 3/10
2953/2953 [==============================] - 208s 70ms/step - loss: 11.7655 - acc: 0.2648
Epoch 4/10
2953/2953 [==============================] - 207s 70ms/step - loss: 10.9550 - acc: 0.3149
Epoch 5/10
2953/2953 [==============================] - 187s 63ms/step - loss: 9.8896 - acc: 0.3786
Epoch 6/10
2953/2953 [==============================] - 194s 66ms/step - loss: 9.3106 - acc: 0.4101
Epoch 7/10
2953/2953 [==============================] - 199s 67ms/step - loss: 4.6355 - acc: 0.6041
Epoch 8/10
2953/2953 [==============================] - 188s 64ms/step - loss: 1.5071 - acc: 0.7701
Epoch 9/10
2953/2953 [==============================] - 191s 65ms/step - loss: 0.7612 - acc: 0.8500
Epoch 10/10
2953/2953 [==============================] - 210s 71ms/step - loss: 0.4170 - acc: 0.